<a href="https://colab.research.google.com/github/helincesxyz/finetuneLLM/blob/main/ConversationalChatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. Install Dependencies

To enable CUDA:
First, you need to ensure that your Colab notebook is set to use a GPU or TPU. Here's how to enable a GPU in Colab:

1. Go to the "Edit" menu and select "Notebook settings" or "Runtime" > "Change runtime type".
2. Choose "GPU" and T4 as the hardware accelerator.
3. Click "Save" to apply the changes.
You can also use Colab Proand choose A100 for faster use.

Please Note:
- Using isolated sandbox systems such as Google Colabs significantly lowers the possibility of conflicts or problems with permissions with the system package manager.

- Generally, without taking certain unusual procedures, **you are not able to construct classic virtual environments in Colab using tools like virtualenv or venv**. It is therefore often safe to proceed with pip installations directly in Colab notebooks, even though the warning
 *(Running pip as the 'root' user can result in broken permissions and conflicting behaviour with the system package manager. It is recommended to use a virtual environment instead: https://pip.pypa.io/warnings/venv)*
 is technically valid.
- The -U flag ensures that if the library is already installed, it will be upgraded to the specified version to prevent any bugs.

In [14]:
!pip install -Uqqq pip
!pip install -qqq bitsandbytes==0.43.0
!pip install -qqq torch==2.2.1
!pip install -qqq -U git+https://github.com/huggingface/transformers
!pip install -qqq -U git+https://github.com/huggingface/peft
!pip install -qqq -U git+https://github.com/huggingface/accelerate
!pip install -qqq datasets==2.12.0
!pip install -qqq loralib==0.1.1
!pip install -qqq einops==0.6.1

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 MB 8.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 18.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done


#1.1 Import Installed Libraries

In [17]:
!git clone https://github.com/TimDettmers/bitsandbytes.git
%cd bitsandbytes/



Cloning into 'bitsandbytes'...
remote: Enumerating objects: 5915, done.
remote: Counting objects: 100% (2017/2017), done.
remote: Compressing objects: 100% (272/272), done.
remote: Total 5915 (delta 1880), reused 1761 (delta 1743), pack-reused 3898
Receiving objects: 100% (5915/5915), 1.79 MiB | 5.53 MiB/s, done.
Resolving deltas: 100% (3998/3998), done.
/content/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes


#FIX ERROR

In [18]:
import json
import os
from pprint import pprint
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset
from huggingface_hub import notebook_login
from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig
)

os.environ["CUDA_VISIBLE_DEVICES"] = "0"

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 4>:4                                                                              │
│                                                                                                  │
│ /content/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes/__init__. │
│ py:6 in <module>                                                                                 │
│                                                                                                  │
│    3 # This source code is licensed under the MIT license found in the                           │
│    4 # LICENSE file in the root directory of this source tree.                                   │
│    5                                                                                             │
│ ❱  6 from . import research, utils                                                               │
│    7 from .autograd._functions import (                                                          │
│    8 │   MatmulLtState,                                                                          │
│    9 │   bmm_cublas,                                                                             │
│                                                                                                  │
│ /content/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes/bitsandbytes/research/ │
│ __init__.py:2 in <module>                                                                        │
│                                                                                                  │
│   1 from . import nn                                                                             │
│ ❱ 2 from .autograd._functions import (                                                           │
│   3 │   matmul_fp8_global,                                                                       │
│   4 │   matmul_fp8_mixed,                                                                        │
│   5 │   switchback_bnb,                                                                          │
│                                                                                                  │
│ /content/bitsandbytes/bitsandbytes/research/autograd/_functions.py:8 in <module>                 │
│                                                                                                  │
│     5                                                                                            │
│     6 import torch                                                                               │
│     7                                                                                            │
│ ❱   8 from bitsandbytes.autograd._functions import GlobalOutlierPooler, MatmulLtState            │
│     9 import bitsandbytes.functional as F                                                        │
│    10                                                      

In [19]:
!pip show bitsandbytes


Name: bitsandbytes
Version: 0.43.0
Summary: k-bit optimizers and matrix multiplication routines.
Home-page: https://github.com/TimDettmers/bitsandbytes
Author: Tim Dettmers
Author-email: dettmers@cs.washington.edu
License: MIT
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, torch
Required-by: 


In [9]:
!pip list


Package                          Version
-------------------------------- ---------------------
absl-py                          1.4.0
accelerate                       0.20.0.dev0
aiohttp                          3.9.3
aiosignal                        1.3.1
alabaster                        0.7.16
albumentations                   1.3.1
altair                           4.2.2
annotated-types                  0.6.0
anyio                            3.7.1
appdirs                          1.4.4
argon2-cffi                      23.1.0
argon2-cffi-bindings             21.2.0
array-record                     0.5.0
arviz                            0.15.1
astropy                          5.3.4
astunparse                       1.6.3
async-timeout                    4.0.3
atpublic                         4.0
attrs                            23.2.0
audioread                        3.0.1
autograd                         1.6.2
Babel                            2.14.0
backcall                         0.2

In [8]:
python -m bitsandbytes

╭──────────────────────────────────────────────────────────────────────────────────────────────────╮
│ python -m bitsandbytes                                                                           │
│           ▲                                                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
SyntaxError: invalid syntax

#1.2 Access HuggingFace API Token

In [20]:
notebook_login()

#2. Upload Falcon 7B

In [21]:
MODEL_NAME = "vilsonrodrigues/falcon-7b-instruct-sharded"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'BitsAndBytesConfig' is not defined

In [22]:
def print_trainable_parameters(model):
  """
  Prints the number of trainable parameters in the model.
  """
  trainable_params = 0
  all_param = 0
  for _, param in model.named_parameters():
    all_param += param.numel()
    if param.requires_grad:
      trainable_params += param.numel()
  print(
      f"trainable params: {trainable_params} || all params: {all_param} || trainables%: {100 * trainable_params / all_param}"
  )

In [ ]:
model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

#3. Deployment Tests

In [ ]:
prompt = """
<human>: what do you do this evening
<assistant>:
""".strip()

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#4. LOG PRIVATE DATASET

In [23]:
from datasets import load_dataset

data = load_dataset("csv", data_files={
    "train": "train.csv",
    "test": "test.csv",
    "validation": "validation.csv"
})


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 3>:3                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/load.py:1773 in load_dataset                    │
│                                                                                                  │
│   1770 │   )                                                                                     │
│   1771 │                                                                                         │
│   1772 │   # Create a dataset builder                                                            │
│ ❱ 1773 │   builder_instance = load_dataset_builder(                                              │
│   1774 │   │   path=path,                                                                        │
│   1775 │   │   name=name,                                                                        │
│   1776 │   │   data_dir=data_dir,                                                                │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/load.py:1502 in load_dataset_builder            │
│                                                                                                  │
│   1499 │   if use_auth_token is not None:                                                        │
│   1500 │   │   download_config = download_config.copy() if download_config else DownloadConfig(  │
│   1501 │   │   download_config.use_auth_token = use_auth_token                                   │
│ ❱ 1502 │   dataset_module = dataset_module_factory(                                              │
│   1503 │   │   path,                                                                             │
│   1504 │   │   revision=revision,                                                                │
│   1505 │   │   download_config=download_config,                                                  │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/load.py:1137 in dataset_module_factory          │
│                                                                                                  │
│   1134 │   │   │   data_files=data_files,                                                        │
│   1135 │   │   │   download_config=download_config,                                              │
│   1136 │   │   │   download_mode=download_mode,                                                  │
│ ❱ 1137 │   │   ).get_module()                                                                    │
│   1138 │   # Try locally                                                                         │
│   1139 │   elif path.endswith(filename):                                                         │
│   1140 │   │   if os.path.isfile(path):                                                          │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/datasets/load.py:712 in get_module                       │
│                                                                                                  │
│    709 │   │   patterns = (                                                                      │
│    710 │   │   │   sanitize_patterns(self.data_files) if self.data_files is not None else get_d  │
│    711 │   │   )                                                                                 │
│ ❱  712 │   │   data_files = DataFilesDict.from_local_or_remote(                                  │
│    713 │   │   │   patterns,                               

In [ ]:
def generate_prompt(data_point):
  return f"""
<human>: {data_point["User"]}
<assistant>: {data_point["Prompt"]}
""".strip()

def generate_and_tokenize_prompt(data_point):
  full_prompt = generate_prompt(data_point)
  tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
  return tokenized_full_prompt

In [ ]:
data = data["train"].shuffle().map(generate_and_tokenize_prompt)

#5. Fine-tuning The Model

In [ ]:
training_args = transformers.TrainingArguments(
      per_device_train_batch_size=1,
      gradient_accumulation_steps=4,
      num_train_epochs=1,
      learning_rate=2e-4,
      fp16=True,
      save_total_limit=3,
      logging_steps=1,
      output_dir="experiments",
      optim="paged_adamw_8bit",
      lr_scheduler_type="cosine",
      warmup_ratio=0.05,
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
model.config.use_cache = False
trainer.train()

#6. Save And Deploy The Model

In [ ]:
model.save_pretrained("trained-model")

In [ ]:
PEFT_MODEL = "jzdesign/midjourney-falcon-7b"

model.push_to_hub(
    PEFT_MODEL, use_auth_token=True
)

In [ ]:
config = PeftConfig.from_pretrained(PEFT_MODEL)
model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    return_dict=True,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True
)

tokenizer=AutoTokenizer.from_pretrained(config.base_model_name_or_path)
tokenizer.pad_token = tokenizer.eos_token

model = PeftModel.from_pretrained(model, PEFT_MODEL)

#6.1 Running The model

In [ ]:
generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

prompt = """
<human>: midjourney prompt for a boy running in the snow
<assistant>:
""".strip()

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
  outputs = model.generate(
      input_ids = encoding.input_ids,
      attention_mask = encoding.attention_mask,
      generation_config = generation_config
  )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))

#6.2 Deployment with Streamlit or FastAPI

In [ ]:
"""

!pip install langchain
from langchain.llms import HuggingFacePipelineLLM
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

# Load the Hugging Face pipeline
llm = HuggingFacePipelineLLM(model_name="meta-llama/Llama-2-7b-chat-hf")

# Define the prompt template
prompt = PromptTemplate(
    input_variables=["context"],
    template=

)

# Create the LLM chain
llm_chain = LLMChain(llm=llm, prompt=prompt)

# Define the context
context = "Hello! I am a conversational AI. How may I assist you today?"

# Generate the response
response = llm_chain.run(context)

# Print the response
print(response)
"""

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 260.9/260.9 kB 23.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.9/70.9 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 24.0
    Uninstalling packaging-24.0:
      Successfully uninstalled packaging-24.0


ImportError: cannot import name 'HuggingFacePipelineLLM' from 'langchain.llms' (/usr/local/lib/python3.10/dist-packages/langchain/llms/__init__.py)